### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Rectal_Cancer/GSE115513'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Checking for gene expression data availability based on given summaries.
is_gene_available = False

# Finding the keys for the variables
# Rectal_Cancer might be inferred from 'organ: Colon' or 'organ: Rectum'
# Age is clearly given with the key 3
# Gender is given with the key 4

import re

# A quick check to determine if 'Rectal_Cancer' can be inferred.
rectal_cancer_values = [v for v in sample_characteristics[1] if 'Rectum' in v]
if len(rectal_cancer_values) > 1:
    trait_row = 1

# Age is available
age_row = 3

# Gender is available
gender_row = 4

# Define conversion functions
def convert_trait(value):
    # Assuming 'organ: Rectum' implies Rectal_Cancer presence
    if 'Rectum' in value:
        return 1
    elif 'Colon' in value:
        return 0
    return None

def convert_age(value):
    match = re.search(r'(\d+)', value)
    if match:
        return int(match.group(1))
    return None

def convert_gender(value):
    if 'Female' in value:
        return 0
    elif 'Male' in value:
        return 1
    return None

# Save Metadata
save_cohort_info('GSE115513', './preprocessed/Rectal_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Rectal_Cancer', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Rectal_Cancer/trait_data/GSE115513.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
